# Equations of State (EOS)

The mechanical state of a substance, as distinguished from a thermal or thermodynamic one, is known when the pressure, temperature, and volume are fixed. These equations are relations between certain groups of three properties like PVT properties. The standard form:

$$P = \frac{RT}{v-b} - \frac{a}{(V+ϵb)(V+σb)}$$

In [24]:
!pip install chemicals
#The chemicals library features an extensive compilation of pure component chemical data

In [25]:
import numpy as np #numpy library for mathematical functions
from plotly.subplots import make_subplots #plotly library for visualizations
import plotly.io as pio
pio.templates.default="plotly_dark"

In [26]:
# First ask the user for a component
def main():
  try:
    from chemicals import CAS_from_any, MW, Tc, Pc, Vc, omega
    component = input("Please enter a compound like (water, butane...): ")
    # extract the component data
    CAS = CAS_from_any(component)
    Pc=Pc(CAS)/101325 #Pa to atm
    Tc=Tc(CAS) #K
    n =MW(CAS) #kmol
    w=omega(CAS) #acentric factor
    R=0.082057   #liter atm/g-mol K
    T=394.2 #K
    Tr = T/Tc
    print(f"This are the pure component chemical data of {component}:")
    print(f"Pc:{Pc:.3f}, Tc:{Tc}, n:{n}, w:{w}, R:{R}, T:{T}, Tr:{Tr:.3f}")
    eq_plot(T,R,Tc,Pc,component,Tr,w)
  except ValueError:
    print(f"Chemical name {component} not recognized")

main()

#EOS parameters
def cubic_abes(T,eos,R,Tc,Pc,Tr,w):
  # The Soave Eq.
  if eos=="SRK":
    sigma = 1
    epsilon= 0
    omega=0.08664
    psi= 0.42748
    alpha = (1+(0.480+1.574*w-0.176*w**2)*(1-Tr**0.5))**2
  # The Peng-Robinson Eq.
  elif eos =="PR":
    sigma = 1 + 2**0.5
    epsilon= 1 - 2**0.5
    omega=0.07780
    psi= 0.45724
    alpha = (1+(0.37464+1.54226*w-0.26992*w**2)*(1-Tr**0.5))**2
  # The van der Waals Eq.
  elif eos =="VDW":
    sigma = 0
    epsilon= 0
    omega=1/8
    psi= 27/64
    alpha = 1
  # The Redlich-Kwong Eq.
  elif eos =="RK":
    sigma = 1
    epsilon= 0
    omega=0.08664
    psi= 0.42748
    alpha = Tr**-0.5
  # repulsion parameter or effective molecular volume
  b= omega*R*Tc/Pc
  # attraction parameter
  a= psi*(alpha*R**2*Tc**2)/Pc
  return a,b,epsilon,sigma

# This function calculates P with the parameters of the seleceted EOS
def cubic_P(v,T,eos,R,Tc,Pc,Tr,w):
  a, b, e, s = cubic_abes(T,eos,R,Tc,Pc,Tr,w)
  return ((R*T)/(v-b))-a/((v+e*b)*(v+s*b))

# This function plot the results of P vs V of each EOS
def eq_plot(T,R,Tc,Pc,component,Tr,w):
  fig =make_subplots()
  eos=["SRK","RK","VDW","PR"]
  for i in eos:
    b= cubic_abes(T,i,R,Tc,Pc,Tr,w)[1]
    v=np.linspace(0.1,2,1000)
    p=cubic_P(v,T,i,R,Tc,Pc,Tr,w)
    fig.add_scatter(x=v,y=p, mode="lines", name=f"{i}")
  fig.update_xaxes(title_text="V (liters/g-mol)",minor_ticks="inside",dtick=0.5)
  fig.update_layout(height=600, width=800, font_family="Arial",
    title={'text': f"Comparison of various state equations for {component} at {T} K",
          'y':0.9,
          'x':0.5,
          'xanchor': 'center',
          'yanchor': 'top'},
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.8))
  v=np.linspace(0.1,2,1000)
  #fig.add_scatter(x=v,y=((R*T)/v), mode="lines",name="Ideal")
  #fig.add_scatter(x=v,y=((R*T)*(v+0.24620*(1-0.09423/v))*(1-350e4/(v*(T**3)))-17.794*(1-0.12161/v))/(v**2), mode="lines",name="BB")
  fig.update_yaxes(range=[0,50], title="P (atm)",minor_ticks="inside", dtick=10)
  fig.show()


Please enter a compound like (water, butane...): butane
This are the pure component chemical data of butane:
Pc:37.464, Tc:425.125, n:58.1222, w:0.201, R:0.082057, T:394.2, Tr:0.927
